Finding the best dynamic RNN model was run on AWS because the average CPU time is 20 min. We did not save the output of that python file, so here we load the saved models, compile them with their weights, and validate that the 32-32 model performs the best on validation data.

In [1]:
%run ../MLHCExperiment-Common-Val17Test18.ipynb

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
all_visits = read_data()
all_visits = all_visits.where(all_visits["AdjCount"] > 0).dropna()

In [3]:
epochs = 300
patience = 5
trials = 1

In [4]:
def run_dynamic_rnn(filename, first_layer_size, second_layer_size, name,
                    features, label):
    print(name)
    X = all_visits[features]
    y = all_visits[label]

    val_mses = list()
    mses = list()
    early_stop = keras.callbacks.EarlyStopping(monitor='val_mean_squared_error',
                                               patience=patience, restore_best_weights=True)

    for j in range(trials):
        start = time.time()
        #print("Trial %d" % (j + 1))

        features_min_max = preprocessing.MinMaxScaler()
        X_train, y_train, X_val, y_val, X_test, y_test = split_for_baseline_and_nn(X, y)

        model = compile_rnn_weights_from_file(filename, X_train, first_layer_size, second_layer_size)

        X_train_t = reshape_for_rnn(features_min_max.fit_transform(X_train))
        X_val_t = reshape_for_rnn(features_min_max.transform(X_val))
        X_test_t = reshape_for_rnn(features_min_max.transform(X_test))
            
        y_val_pred = model.predict(X_val_t, batch_size=1)
        y_pred = model.predict(X_test_t, batch_size=1)
        val_mses.append(mean_squared_error(y_val_pred, y_val))
        mses.append(mean_squared_error(y_pred, y_test))
        end = time.time()
        print("Trial completed in %.2f s" % (end - start))
    
    return val_mses, mses, y_pred

In [7]:
best_model = "64-64"
min_v_mse = 9999
mse_dct = dict()

for arch in possible_architectures:
    print(arch)
    filenames = prebuilt_rnn_models[arch]

    v_mses = list()
    mses = list()
    for filename in filenames:
        v_mse, mse, _ = run_dynamic_rnn("../" + filename, layer_sizes[arch][0],
                                          layer_sizes[arch][1], filename, BASE_FEATURES,
                                          ALL_VISITS_LABEL)
        v_mses.append(v_mse)
        mses.append(mse)
    
    print_mse_metrics(v_mses)
    mse_dct[arch] = mses
    if np.mean(v_mses) < min_v_mse:
        min_v_mse = np.mean(v_mses)
        best_model = arch
    print("Best model %s" % (best_model))
    print("Min Val MSE %.5f" % (min_v_mse))

print("Best model overall is %s with avg test MSE %.3f" % (best_model, np.mean(mse_dct[best_model])))

64-64
RNNModels/64-64/Dynamic-RNN-AllVisits-64-64-Trial3-1562305234.6405256.h5
Trial completed in 6.14 s
RNNModels/64-64/Dynamic-RNN-AllVisits-64-64-Trial1-1562303707.4653575.h5
Trial completed in 6.56 s
RNNModels/64-64/Dynamic-RNN-AllVisits-64-64-Trial4-1562305849.0060897.h5
Trial completed in 6.35 s
RNNModels/64-64/Dynamic-RNN-AllVisits-64-64-Trial2-1562304439.9532201.h5
Trial completed in 7.31 s
RNNModels/64-64/Dynamic-RNN-AllVisits-64-64-Trial0-1562302921.312181.h5
Trial completed in 7.21 s
MSE and RMSE over 10 trials with standard deviation in parentheses
Average MSE: 0.717 (0.005)
Average RMSE: 0.847 (0.003)
Best model 64-64
Min Val MSE 0.71734
64-32
RNNModels/64-32/Dynamic-RNN-AllVisits-64-32-Trial3-1562309408.0298276.h5
Trial completed in 7.62 s
RNNModels/64-32/Dynamic-RNN-AllVisits-64-32-Trial4-1562310198.6327813.h5
Trial completed in 7.87 s
RNNModels/64-32/Dynamic-RNN-AllVisits-64-32-Trial1-1562307769.6175947.h5
Trial completed in 8.64 s
RNNModels/64-32/Dynamic-RNN-AllVisits-